# Ambiente Lagartija

In [ ]:
HEADER = '\033[95m'
OKBLUE = '\033[94m'
OKCYAN = '\033[96m'
OKGREEN = '\033[92m'
WARNING = '\033[93m'
FAIL = '\033[91m'
ENDC = '\033[0m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'

In [ ]:
import random
from IPython.display import clear_output
from time import sleep
random.seed(10)

In [ ]:
class Ambiente:
    def __init__(self, size = (3,3), cricket = 1, cricket_5 = 1, bird = 1):
        self.agent_score = 0 
        self.size = size
        self.cricket = cricket
        self.cricket_5 = cricket_5
        self.bird = bird
        self.end = False
        self.inicial_states = {}
        self.agent_x = None
        self.agent_y = None
#####################################Crear Mapa#####################################
        # self.map = [[(-1 , "empty") for i in range(size[0])] for e in range(size[1])]
        # for c in range(self.cricket):
        #     x, y = self.random_empty_cell()
        #     self.map[y][x] = (5, f"cricket_{c}")

        # for c in range(self.cricket_5):
        #     x, y = self.random_empty_cell()
        #     self.map[y][x] = (10, f"c5_{c}")
        # for b in range(self.bird):
        #     x, y = self.random_empty_cell()
        #     self.map[y][x] = (-10, f"bird_{b}")
        self.map = [
            [(10, f"5_crickets"), (-1 , "empty"), (5, f"cricket")],
            [(-1 , "empty"), (-10, f"bird"), (-1 , "empty")],
            [(-1 , "empty"), (-1 , "empty"), (-1 , "empty")]
        ]
        self.agent_x, self.agent_y = self.random_empty_cell()
        counter = 0
        for y in range(len(self.map)):
            for x in range(len(self.map[y])):
                if self.map[y][x][0] == -1:
                    self.map[y][x] = (-1, f"empty_{counter}")
                    counter +=1
        self.inicial_states = self.__dict__.copy()
                    
    def random_empty_cell(self):
        extend = [x[0] for y in self.map for x in y ]
        
        if any(x == -1 for x in extend):
            positions = [(x,y) for x in range(len(self.map)) for y in range(len(self.map[0]))]
            found = False
            x = 0
            y = 0
            while not found:
                x, y = positions.pop(random.randint(0, len(positions)-1))
                if self.map[y][x][0] == -1:
                        found = True
            return x,y
        else:
            raise Exception("Ninguna celda disponible")
            
###################################################################################


    def action(self, action):
        reward, done, score = (None, False, None)
        if not self.end:
            if abs(self.map[self.agent_y][self.agent_x][0]) == 10:
                # print("End")
                self.end = True
                reward = self.map[self.agent_y][self.agent_x][0]
                score = self.agent_score + reward

            if action == "Up":
                if self.agent_y == 0: #Revisar si la movida es válida
                    reward = -2
                    score = self.agent_score
                else:
                    # Actualiza score y posición del agente
                    self.agent_score += self.map[self.agent_y-1][self.agent_x][0]
                    reward = self.map[self.agent_y-1][self.agent_x][0]
                    self.agent_y -=1
                    score = self.agent_score
            if action == "Down":
                if self.agent_y == len(self.map) -1:
                    reward = -2
                    score = self.agent_score
                else:
                    self.agent_score += self.map[self.agent_y+1][self.agent_x][0]
                    reward = self.map[self.agent_y+1][self.agent_x][0]
                    self.agent_y +=1
                    score = self.agent_score
            if action == "Left":
                if self.agent_x == 0:
                    reward = -2
                    score = self.agent_score
                else:
                    self.agent_score += self.map[self.agent_y][self.agent_x-1][0]
                    reward = self.map[self.agent_y][self.agent_x-1][0]
                    self.agent_x -=1
                    score = self.agent_score
            if action == "Right":
                if self.agent_x == len(self.map[0]) -1:
                    reward = -2
                    score = self.agent_score
                else:
                    self.agent_score += self.map[self.agent_y][self.agent_x +1 ][0]
                    reward = self.map[self.agent_y][self.agent_x +1 ][0]
                    self.agent_x +=1
                    score = self.agent_score
        else:
            print("Ha terminado el juego")
        done = self.end
        return (reward, done, score)
        
    def reset(self):
        # print(self.inicial_states)
        self.__dict__ = self.inicial_states
        self.inicial_states = self.__dict__.copy()
    def show_map(self):
        #Print amigable del mapa
        state = self.get_state()
        string = f'Celda actual: {state[0]}'
        for y_i, y in enumerate(self.map):
            string += '\n'
            for x_i, x in enumerate(y):
                a = '|'+ f'{x[1]}'.center(11) + '|'
                if x_i == state[1] and y_i == state[2]:
                    a = '|'+ BOLD + f'{x[1]}'.center(11) + ENDC + '|'
                string += a
        return string
    def get_state(self):
        #Retorna celda actual
        return (self.map[self.agent_y][self.agent_x][1], self.agent_x, self.agent_y)
    
    def get_score(self):
        #Retornal score actual
        return self.agent_score
        

In [ ]:
import random
import numpy as np
import pandas as pd
from tabulate import tabulate

LR = 0.1
DISCOUNT_RATE = 0.99
MAX_EXPLORATION_RATE = 1
MIN_EXPLORATION_RATE = 0.0001
EXPLORATION_DECAY_RATE = 0.01

class Agent:

    def __init__(self):
        self.q_table = np.zeros((3*3, 4))
        self.n_games = 0
        self.EXPLORATION_RATE = MIN_EXPLORATION_RATE + (MAX_EXPLORATION_RATE - MIN_EXPLORATION_RATE) * np.exp(-EXPLORATION_DECAY_RATE*self.n_games)

    def get_state(self, env):
        _, x, y = env.get_state()
        return 3*y + x

    def get_action(self, state):
        move = 0
        if random.uniform(0, 1) < self.EXPLORATION_RATE: # Exploramos
            move = random.randint(0, 3)
        else: # Explotamos
            # Si los valores para este estado siguen en 0, tomamos una acción random para no sesgar al agente.
            if not np.any(self.q_table[state,:]): 
                move = random.randint(0, 3)
            else: 
                move = np.argmax(self.q_table[state,:])
        return move
    
    def print_qtable(self, env):
        states = np.array([item[1] for sublist in env.map for item in sublist])
        table = np.c_[ states, self.q_table ].tolist() 
        print(tabulate(table, headers=["State", "Up", "Down", "Left", "Right"], tablefmt='orgtbl'))

def train(NUM_EPISODES = 100, show=True, agent=None):
    if not agent:
        agent = Agent()
    env = Ambiente()
    moves = ["Up", "Down", "Left", "Right"]
    while True:
        if show:
            clear_output()
            print(env.show_map())
            print()
            agent.print_qtable(env)
        state = agent.get_state(env)
        move = agent.get_action(state)
        reward, done, score = env.action(moves[move])
        state_new = agent.get_state(env)
        agent.q_table[state, move] = agent.q_table[state, move] * (1 - LR) + \
            LR * (reward + DISCOUNT_RATE * np.max(agent.q_table[state_new,:]))
        if done:
            if show:
                print("END")
            agent.EXPLORATION_RATE = MIN_EXPLORATION_RATE + \
                (MAX_EXPLORATION_RATE - MIN_EXPLORATION_RATE) * np.exp(-EXPLORATION_DECAY_RATE*agent.n_games)
            env.reset()
            agent.n_games += 1 
            if agent.n_games == NUM_EPISODES:
                break
        if show:
            print(f"Nos movemos {moves[move]}")
            print(f"Recibimos {reward}")
            input()
    return (agent, env)

def test(agent=None, env=None, show=True):
    if not agent or not env:
        print("Se necesita un agente y un ambiente")
    moves = ["Up", "Down", "Left", "Right"]
    while True:
        if show:
            clear_output()
            print(env.show_map())
            agent.print_qtable(env)
        state = agent.get_state(env)
        move = np.argmax(agent.q_table[state,:])
        reward, done, score = env.action(moves[move])
        state_new = agent.get_state(env)
        if done:
            if show:
                print("END")
            env.reset()
            agent.n_games += 1 
        if show:
            print(f"Nos movemos {moves[move]}")
            print(f"Recibimos {reward}")
            end = input()
            if end:
                break


In [ ]:
agent, env = train(show=False)

In [14]:
test(agent=agent, env=env)

Celda actual: empty_1
| 5_crickets||  empty_0  ||  cricket  |
|  empty_1  ||    bird   ||  empty_2  |
|  empty_3  ||  empty_4  ||  empty_5  |
| State      |       Up |      Down |      Left |     Right |
|------------+----------+-----------+-----------+-----------|
| 5_crickets |  7.50027 | 22.2442   |  3.32102  |  0.112804 |
| empty_0    |  0       |  0        |  1.18504  |  0        |
| cricket    |  0       |  0        | -0.1      |  0        |
| empty_1    | 28.3998  |  8.3328   | 13.7883   | -7.12843  |
| bird       | -0.35124 | -0.61609  |  7.01225  | -0.136855 |
| empty_2    |  0.5     | -0.113909 |  0        |  0        |
| empty_3    | 17.0401  |  1.95482  | -0.570923 | -0.534309 |
| empty_4    | -2.71    | -0.288205 |  2.48209  | -0.3439   |
| empty_5    | -0.1405  | -0.38     | -0.184726 | -0.2      |
Nos movemos Up
Recibimos 10


KeyboardInterrupt: ignored